Librerias a trabajar con Google Drive & Google Sheets

In [4]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread
import pprint
import pandas as pd
import numpy as np

In [5]:
#Documentación
ruta_accesos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"
alcances = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#autenticación
credenciales = service_account.Credentials.from_service_account_file(ruta_accesos,scopes = alcances) 

#construir el servicio de google drive
drive = build('drive', 'v3', credentials= credenciales)

# Ruta al archivo JSON de credenciales de la cuenta de servicio
ruta_accesos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"
alcances = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Autenticación
credenciales = service_account.Credentials.from_service_account_file(ruta_accesos, scopes=alcances)

# Construir el servicio de Google Drive
drive = build('drive', 'v3', credentials=credenciales)

In [40]:
#Funcion de lectura de documentos de google drive:
def lista_carpetas(service):
    query = "mimeType='application/vnd.google-apps.folder'" #tipo de archivo de google Drive (Multipurpose Internet Mail Extensions-MIME)
    resultados = service.files().list(q=query).execute()
    carpetas = pd.DataFrame(resultados.get('files', []))
    
    return carpetas[['name','id']]
carpetas = lista_carpetas(drive)
dimh = len(carpetas)
print(carpetas.iloc[:,:])


                 name                                 id
0        Cotizaciones  1Zi0tlWbPx62WV5i37FeFY85ogSOyJ0QD
1   Archivos antiguos  1fT3E2QXncZI3NFgD44AMnWNdV9bmNXoK
2                Mayo  1Dp5Wj2elCNjg4hWIrbXpOHvlPBZq8Mx9
3               Abril  1LEujdENDFRY3NMX0Y2Bmu54gYVV9oo_U
4               Marzo  150Wit_6ex4R1GGcAx5Nk5xZzmnCVon6l
5             Febrero  19NwdKE1MLORKJis13c4MWfO0mwx5oBqf
6               Enero  1_RY_xRX3ccMhGCTrxvolevlgxOLOsg-j
7           Diciembre  11oEIorz34jLdqUziLndNMFHbYXrE4rYU
8           Noviembre  17xVkgol8_iXUwWe4r6hpN3i2sLYAtYvM
9             Octubre  1PGup0YDJzxTZqvb5YkDX2QLdQjhOVZVF
10         Septiembre  1Qi6z9HSCUsdfGHocCYFuODUBDdRPC2J1
11             Agosto  1A_GY9igGyRDGKz2zK7hvOgtVwCODcSq8
12              Julio  1P4LPctBcaYdE3kzgO8saNz-K6Ict5qpX
13              Junio  1-w1je13MvWX5UorwjLJLoFpwBnqxsxKi
14               Mayo  1iQNYxIMCDQGDlb_qgRd2RVz18CUbyb6l
15              Abril  1DTwJqtnypPlyYCUa94MUW3s-nXDtN40G
16              Marzo  1PG2riEz

In [42]:
#Generamos los encabezados a partir de los nombres de las carpetas:
h = np.full(dimh, " ", dtype='<U50')
for j in range(dimh):
    h[j] = carpetas.iloc[j,0]
print(h)    

['Cotizaciones' 'Archivos antiguos' 'Mayo' 'Abril' 'Marzo' 'Febrero'
 'Enero' 'Diciembre' 'Noviembre' 'Octubre' 'Septiembre' 'Agosto' 'Julio'
 'Junio' 'Mayo' 'Abril' 'Marzo' 'Febrero' 'Enero' '2023' '2024']


In [44]:
#funcion para extraer los nombres de los archivos dentro de las carpetas
def archivos_en_carpeta(service, folder_id):
    query = f"'{folder_id}' in parents"
    
    resultados = service.files().list(q=query).execute()
    #pprint.pprint(resultados)  # Imprime la respuesta completa para depuración
    datos = (pd.DataFrame(resultados.get('files', [])))[['name']]
    return datos
            
#Guardar los nombres de los archivos en un objeto dataframe, la columna es la carpeta y las filas los archivos
#print(archivos_en_carpeta(drive,carpetas.iloc[1,1]))

datos = pd.DataFrame(np.zeros((len(carpetas),len(carpetas.head()))))
for j in range(len(carpetas.head())):
    for i in range(len(carpetas)):
        #print(f"esto proviene de la segunda funcion: {archivos_en_carpeta(drive,carpetas.iloc[i,1])}")
        #print(f"esto proviene de la primera funcion: {carpetas.iloc[j,0]}")
        datos.iloc[i,j] = archivos_en_carpeta(drive,carpetas.iloc[i,1])
#datos.columns = h       
print(datos)

                                                    0  \
0   0    RELACION OC-FACTURAS
1                   ...   
1   0                                       formul...   
2   0     CTQ-1249 ZF PLANTA 1 (Servicio de Descar...   
3   0     CTQ-1236 EATON BUSSMANN (RENTA DE PLATAF...   
4   0     CTQ-1192 ZF Santa Rosa (conexion electri...   
5   0     CTQ-1159 EATON BUSSMANN (SERVICIO PARA P...   
6   0     CTQ-1130 EATON BUSSMANN ( SERVICIO DE FA...   
7   0     CTQ-1092 EATON BUSSMANN (SERVICIO DE TIE...   
8   0     CTQ-1036 EATON COOPER (SERVICIO DE INSTA...   
9   0     CTQ-10023 ZF (SERVICIO DE BAJADAS ELÉCTR...   
10  0     CTQ-1011 EATON BUSSMANN (SERVICIO DE INS...   
11  0     CTQ-983 ZF (Reubicación, Conexion de equ...   
12  0     CTQ-958 EATON BUSSMANN (SERVICIO DE INST...   
13  0     CTQ-948 EATON BUSSMANN (RENTA DE PLATAFO...   
14  0     CTQ-921 SAARGUMMI (FABRICACIÓN DE BASES ...   
15  0     CTQ-902 ZF(Maniobra desembalaje, puesta ...   
16  0     CTQ-885 EATON BUSSMAN